In [1]:
from opt.mc_sim import *
from common.variables import *
from sim.sim_functions import *
import torch
import gym
from gym import spaces
from gym import Env
from gym.spaces import Discrete, Box, MultiDiscrete, Tuple, MultiBinary

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
s = SourcingEnv()

In [3]:
class CustomGymEnv(Env):
    #metadata = {'render.modes': ['human']}  
    
    def __init__(self, sourcing_env):
        self.SourcingEnv = sourcing_env
        self.counter = 0
        
        # Actions we can take, down, stay, up
        self.action_space = MultiDiscrete([30,30])

        # Inventory Observation State
        self.observation_space = Box(low=np.array([-30, 0, 0, 0, 0]), high=np.array([30, 30, 30, 1, 1]), shape=(5,), dtype=int)
        #Tuple(Box(-30,30,shape=(1,), dtype=int), Discrete(30), Discrete(30), MultiBinary(2))
                                       
    
    def step(self, action):        
        reward = self.reward_func(self.SourcingEnv.current_state, action)
        next_state, event, i, event_probs, supplier_index = self.SourcingEnv.step(action)
        self.counter += 1
        
        info = {}
        
        if self.counter < PERIODS:
            done = False
        else:
            done = True
        
        next_state_array = np.array(next_state.get_list_repr())
        return next_state_array, reward, done, info
    
    def reset(self):
        self.SourcingEnv = SourcingEnv()
        return np.array(self.SourcingEnv.current_state.get_list_repr())
        
    def reward_func(self, state, action):
        reward_hb = H_COST * state.s if state.s >= 0 else  -B_PENALTY * state.s 
        reward = reward_hb + np.sum(np.multiply(action, PROCUREMENT_COST_VEC))
        reward = float(reward)
        return -reward
    
    #def render(self):
        # Implement viz
       # pass

In [4]:
s = SourcingEnv()

In [5]:
custom_gym_env = CustomGymEnv(s)

In [6]:
m_state = custom_gym_env.SourcingEnv.current_state
m_state.get_list_repr()

[0, 0, 0, 1, 1]

In [7]:
str(m_state)

'Stock: 0, n backorders: [0. 0.], supplier status (on/off): [1. 1.]'

In [8]:
from stable_baselines3.common.env_checker import check_env

In [9]:
check_env(custom_gym_env) #warn=True)

In [10]:
episodes = 15
for episode in range(1, episodes+1):
    state = custom_gym_env.reset()
    done = False
    cost = 0
    
    
    while not done:
        #env.render()
        action = custom_gym_env.action_space.sample()
        n_state, reward, done, info = custom_gym_env.step(action)
        cost+=reward
        observation = custom_gym_env.step(action)
    print('Episode:{} Cost:{} Observation {}' .format(episode, cost, observation))
    

custom_gym_env.close()

Episode:1 Cost:-1055.0 Observation (array([23, 23,  8,  1,  1]), -1055.0, True, {})
Episode:2 Cost:-1275.0 Observation (array([26, 26, 42,  1,  1]), -1275.0, True, {})
Episode:3 Cost:-1110.0 Observation (array([23, 23, 30,  1,  1]), -1110.0, True, {})
Episode:4 Cost:-1195.0 Observation (array([26, 26, 10,  1,  1]), -1195.0, True, {})
Episode:5 Cost:-780.0 Observation (array([17, 17,  6,  1,  1]), -780.0, True, {})
Episode:6 Cost:-105.0 Observation (array([-2,  2, 24,  1,  1]), -115.0, True, {})
Episode:7 Cost:-810.0 Observation (array([27, 30, 27,  1,  1]), -810.0, True, {})
Episode:8 Cost:0.0 Observation (array([-1,  0,  0,  0,  1]), -10.0, True, {})
Episode:9 Cost:-660.0 Observation (array([14, 14, 12,  1,  1]), -660.0, True, {})
Episode:10 Cost:-540.0 Observation (array([12, 12,  0,  1,  1]), -540.0, True, {})
Episode:11 Cost:-115.0 Observation (array([ 1,  2, 10,  1,  1]), -125.0, True, {})
Episode:12 Cost:-515.0 Observation (array([13, 20, 13,  1,  1]), -515.0, True, {})
Episode:1

In [11]:
custom_gym_env.observation_space

Box([-30   0   0   0   0], [30 30 30  1  1], (5,), int64)

In [12]:
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [13]:
log_path = os.path.join('Training', 'Logs')

In [25]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [26]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_6
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1        |
|    ep_rew_mean     | -750     |
| time/              |          |
|    fps             | 756      |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 100      |
---------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -438      |
| time/                   |           |
|    fps                  | 807       |
|    iterations           | 2         |
|    time_elapsed         | 0         |
|    total_timesteps      | 200       |
| train/                  |           |
|    approx_kl            | 0.5092664 |
|    clip_fraction        | 0.798     |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.61     |
|    explained_variance   | 0         |
|    learning_rate        | 0

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 865       |
|    iterations           | 12        |
|    time_elapsed         | 1         |
|    total_timesteps      | 1200      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.89e-05 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 4.85      |
|    n_updates            | 110       |
|    policy_gradient_loss | 0.975     |
|    value_loss           | 8.49      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 867       |
|    iterations           | 22        |
|    time_elapsed         | 2         |
|    total_timesteps      | 2200      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -1.93e-05 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.0439    |
|    n_updates            | 210       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.121     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 876       |
|    iterations           | 32        |
|    time_elapsed         | 3         |
|    total_timesteps      | 3200      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.01e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.0435    |
|    n_updates            | 310       |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.121     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 886       |
|    iterations           | 42        |
|    time_elapsed         | 4         |
|    total_timesteps      | 4200      |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.01e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | -0.702    |
|    n_updates            | 410       |
|    policy_gradient_loss | -0.746    |
|    value_loss           | 0.121     |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


...

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 901       |
|    iterations           | 952       |
|    time_elapsed         | 105       |
|    total_timesteps      | 95200     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.76e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.0446    |
|    n_updates            | 9510      |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.12      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 901       |
|    iterations           | 962       |
|    time_elapsed         | 106       |
|    total_timesteps      | 96200     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.58e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | -0.701    |
|    n_updates            | 9610      |
|    policy_gradient_loss | -0.746    |
|    value_loss           | 0.12      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 901       |
|    iterations           | 972       |
|    time_elapsed         | 107       |
|    total_timesteps      | 97200     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.27e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.0446    |
|    n_updates            | 9710      |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.12      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 901       |
|    iterations           | 982       |
|    time_elapsed         | 108       |
|    total_timesteps      | 98200     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.43e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.0446    |
|    n_updates            | 9810      |
|    policy_gradient_loss | 0         |
|    value_loss           | 0.12      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |
| time/                   |           |
|    fps                  | 901       |
|    iterations           | 992       |
|    time_elapsed         | 110       |
|    total_timesteps      | 99200     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -6.31e-07 |
|    explained_variance   | nan       |
|    learning_rate        | 0.1       |
|    loss                 | 0.791     |
|    n_updates            | 9910      |
|    policy_gradient_loss | 0.746     |
|    value_loss           | 0.12      |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 1         |
|    ep_rew_mean          | -5        |


In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.1, device='auto', batch_size=10, n_steps=10)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.1, device='auto', batch_size=100, n_steps=100)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.1, device='auto', batch_size=1000, n_steps=1000)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.001, device='auto', batch_size=10, n_steps=10)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.001, device='auto', batch_size=100, n_steps=100)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.001, device='auto', batch_size=1000, n_steps=1000)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.00001, device='auto', batch_size=10, n_steps=10)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.00001, device='auto', batch_size=100, n_steps=100)

In [ ]:
model.learn(total_timesteps=100000)

In [ ]:
model = PPO("MlpPolicy", custom_gym_env, verbose=1, tensorboard_log=log_path, learning_rate=0.00001, device='auto', batch_size=1000, n_steps=1000)

In [ ]:
model.learn(total_timesteps=100000)

In [16]:
model.save('PPO')

In [17]:
evaluate_policy(model, custom_gym_env, n_eval_episodes=10, render=False)

/usr/local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-1055.0, 0.0)

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu
